In [1]:
import numpy as np
import pandas as pd
import re
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

# Loading the data
train = pd.read_csv('trainDataset.csv')
test = pd.read_csv('testDataset.csv')



# Showing overview of the train dataset
train.head(20)

,Survived,PassengerClass,Sex,Age,SiblingSpouse
0,0,3,male,adult,1
1,1,1,female,adult,1
2,1,3,female,adult,0
3,1,1,female,adult,1
4,0,3,male,adult,0
5,0,1,male,adult,0
6,0,3,male,child,1
7,1,3,female,adult,0
8,1,2,female,teenage,1
9,1,3,female,child,1


In [2]:
%matplotlib inline


data = pd.read_csv("trainDataset.csv")
print(data.info())

data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Survived        200 non-null    int64 
 1   PassengerClass  200 non-null    int64 
 2   Sex             200 non-null    object
 3   Age             200 non-null    object
 4   SiblingSpouse   200 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 7.9+ KB
None


,Survived,PassengerClass,SiblingSpouse
count,200.000000,200.000000,200.000000
mean,0.340000,2.355000,0.355000
std,0.474898,0.788724,0.479714
min,0.000000,1.000000,0.000000
25%,0.000000,2.000000,0.000000
50%,0.000000,3.000000,0.000000
75%,1.000000,3.000000,1.000000
max,1.000000,3.000000,1.000000


In [3]:
train=train.replace('child',0).replace('teenage',1).replace('adult',2)
train=train.replace('male',1).replace('female',0)

In [4]:
train

,Survived,PassengerClass,Sex,Age,SiblingSpouse
0,0,3,1,2,1
1,1,1,0,2,1
2,1,3,0,2,0
3,1,1,0,2,1
4,0,3,1,2,0
...,...,...,...,...,...
195,0,3,1,2,0
196,0,1,1,2,1
197,0,3,0,2,0
198,1,2,0,2,0


In [5]:
test=test.replace('child',0).replace('teenage',1).replace('adult',2)
test=test.replace('male',1).replace('female',0)
test

,Survived,PassengerClass,Sex,Age,SiblingSpouse
0,0,3,0,2,0
1,1,3,0,2,0
2,1,1,0,2,0
3,1,1,0,2,0
4,1,2,0,2,0
5,1,3,1,0,1
6,0,1,1,2,1
7,0,1,1,2,0
8,0,2,1,2,0
9,0,3,1,1,1


In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

In [7]:
y_train = train.pop('Survived')


In [8]:
y_train

0      0
1      1
2      1
3      1
4      0
      ..
195    0
196    0
197    0
198    1
199    1
Name: Survived, Length: 200, dtype: int64

In [9]:
train

,PassengerClass,Sex,Age,SiblingSpouse
0,3,1,2,1
1,1,0,2,1
2,3,0,2,0
3,1,0,2,1
4,3,1,2,0
...,...,...,...,...
195,3,1,2,0
196,1,1,2,1
197,3,0,2,0
198,2,0,2,0


In [10]:
y_test = test.pop('Survived')

In [11]:
test

,PassengerClass,Sex,Age,SiblingSpouse
0,3,0,2,0
1,3,0,2,0
2,1,0,2,0
3,1,0,2,0
4,2,0,2,0
5,3,1,0,1
6,1,1,2,1
7,1,1,2,0
8,2,1,2,0
9,3,1,1,1


# HUNT

In [ ]:
reg2 = DecisionTreeClassifier()
reg2.fit(train, y_train)
y_pred = reg.predict(test)

In [ ]:

def stop_condition(labels):
        x = Counter(labels)
        if len(x) > 1: 
            return False
        else:
            return True

""" best_split takes as input 'records' : 2D list of records
    returns attribute with lowest gini split as a numeric value in the range of 0 to # of attributes - 1 """

def best_split(records):
        # n : number of attributes
        # gsplits: list of calculated gini splits for all attributes
        n = len(records[0]) - 1
        d = len(records)
        label_vals = [records[y][-1] for y in range(len(records))] # get label values 
        gsplits = []
        
        # iterate over all attributes to calculate their gsplits
        for i in range(n):
            att_vals = [records[j][i] for j in range(len(records))] # get attribute values of ith attribute
            C = Counter(att_vals)  # get counts of all unique attribute values
            gini_vals = [] # list to store gini index values of all values of a particular attribute
            nr_subsets = [] # list to store number of records in each subset of attribute values
                
            # partition the data by each unique attribute value while getting labels into a list of 2D lists where elements are [att_value, label_value]
        # calculate gini values of each attribute value
            for attribute in C.keys():
                att_subset = [ [att_vals[u], label_vals[u]] for u in range(len(att_vals)) if att_vals[u] == attribute ]
                nr = len(att_subset)
                nr_subsets.append(nr)
                labels_of_subset = [ att_subset[u][1] for u in range(len(att_subset)) ]
                local_c = Counter(labels_of_subset)
                gini = 1 - sum((v/nr)**2 for v in local_c.values())  # p(label) = v/nr
                gini_vals.append(gini)
            
            gs = sum((nr_subsets[x]/d) * gini_vals[x] for x in range(len(nr_subsets)))
            gsplits.append(gs)
        
        return gsplits.index(min(gsplits))


""" Function Build_Tree takes in list of records(type 2D list) and list of 
    attributes(type 1D list) and returns a root of decision tree (type Node) """

def build_tree(records, attributes):
        root = Node()
        labels = [r[-1] for r in records]
        
        if stop_condition(labels):
            root.set_data("Label " + str(labels[0]))
            return root
        
        if attributes.count(0) == 0: # majority voting
            c = Counter(labels)
            majority = c.most_common(1)[0][0] # since most_common returns a list of (element,count) tuples
            root.set_data("Label " + str(majority))
            return root
        
        split_attribute = best_split(records)
        root.set_data("Attribute " + str(split_attribute))
        attributes[split_attribute] = 1
        split_attribute_vals = [records[i][split_attribute] for i in range(len(records))]
        val_counts = Counter(split_attribute_vals)
        for val in val_counts.keys():
            partition = [records[i] for i in range(len(records)) if records[i][split_attribute] == val]
            if len(partition) == 0:
                c = Counter(labels)
                majority = c.most_common(1)[0][0]
                leaf = Node("Label " + str(majority))
                root.add_child(leaf)
            else:
                child = build_tree(partition, attributes)
                root.add_child(child)
        
        return root
        




# GINI 

In [32]:
reg = DecisionTreeClassifier(criterion="gini")
reg.fit(train, y_train)
y_pred = reg.predict(test)

In [33]:
y_pred

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1],
      dtype=int64)

In [34]:
reg

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [26]:
submit = pd.read_csv("sample_submit.csv")
submit['y'] = y_pred
submit.to_csv('my_DT_prediction.csv', index=False)

In [35]:
y_res=[0,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,1,0,0,1]

In [41]:
cnt=0
for i in range(20):
    if(y_pred[i]==y_res[i]):
        cnt=cnt+1
print(cnt)

15


In [43]:
score=float(cnt/20)
print("the score the tree model is : {}".format(score))

the score the tree model is : 0.75
